# 05 Tooling | 01 Semantic Kernel | 01 Plugin

## Azure Environment

Necessary parameter are imported from [./Configuration/application.env]. Check [Create Environment](../../01_CreateEnvironment/01_Environment.ipynb) to setup the necessary demo environment.

This notebooks highlights how Plugins work. 

## Step 1: Create Semantic Kernel

MS Semantic Kernel is a tool or library developed by Microsoft to abstract away communication with LLMs and other services like vector DBs. The nuget package Microsoft.SemanticKernel provides developers with access to this powerful tool, allowing them to integrate advanced language processing capabilities into their applications effortlessly.


In [1]:
#r "nuget: Microsoft.SemanticKernel, 1.15.1"
#r "nuget: DotNetEnv, 2.5.0"

using Microsoft.SemanticKernel;
using Microsoft.SemanticKernel.Connectors.OpenAI; 
using Microsoft.Extensions.Logging;
using Microsoft.Extensions.Logging.Abstractions;
using Microsoft.Extensions.DependencyInjection;
using Kernel = Microsoft.SemanticKernel.Kernel;
using DotNetEnv;
using System.IO;

//configuration file is created during environment creation
static string _configurationFile = @"../../Configuration/application.env";
Env.Load(_configurationFile);

string oAiApiKey = Environment.GetEnvironmentVariable("WS_AOAI_APIKEY") ?? "WS_AOAI_APIKEY not found";
string oAiEndpoint = Environment.GetEnvironmentVariable("WS_AOAI_ENDPOINT") ?? "WS_AOAI_ENDPOINT not found";
string chatCompletionDeploymentName = Environment.GetEnvironmentVariable("WS_CHATCOMPLETION_DEPLOYMENTNAME") ?? "WS_CHATCOMPLETION_DEPLOYMENTNAME not found";
string assetsFolder = Environment.GetEnvironmentVariable("WS_ASSETS_FOLDER") ?? "WS_ASSETS_FOLDER not found";

Kernel kernel = Kernel
    .CreateBuilder()
    .AddAzureOpenAIChatCompletion(
        chatCompletionDeploymentName,
        oAiEndpoint,
        oAiApiKey,
        serviceId: chatCompletionDeploymentName 
    )
    .Build();

Console.WriteLine($"Semantic Kernel created...");


Installed Packages DotNetEnv, 2.5.0 Microsoft.SemanticKernel, 1.15.1

Semantic Kernel created...


## Step 2: Define Plugin

The code cell defines a so called plugin. A plugin is functionality (both native or semantic) which can be executed by Semantic Kernel. In this sample a summarization plugin is defined.



In [2]:
string skPrompt = """
    {{$wikiSuperBowl}}

    Who won the Super Bowl in 2024?
    """;

OpenAIPromptExecutionSettings executionSettings = new OpenAIPromptExecutionSettings 
{
    MaxTokens = 2000,
    Temperature = 0.2,
    TopP = 0.5
};

PromptTemplateConfig promptTemplateConfig = new PromptTemplateConfig(skPrompt);

KernelPromptTemplateFactory promptTemplateFactory = new KernelPromptTemplateFactory();
IPromptTemplate promptTemplate = promptTemplateFactory.Create(promptTemplateConfig);
KernelFunction queryFunction = kernel.CreateFunctionFromPrompt(skPrompt, executionSettings);

Console.WriteLine($"Kernel plugin defined...")

Kernel plugin defined...


## Step 3: Execute Plugin

The Semantic Kernel function `InvokeAsync()` executes the defined plugin and therefore summarizes the provided ebook.

In [3]:
string wikiSuperBowl = Path.Combine(assetsFolder, "Embedding", "WikiSuperBowl2024.txt");

string groundingInformation = await File.ReadAllTextAsync(wikiSuperBowl);
var summaryResult = await kernel.InvokeAsync(queryFunction, new() { ["wikiSuperBowl"] = groundingInformation });

Console.WriteLine(summaryResult);


The Kansas City Chiefs won the Super Bowl LVIII in 2024. They defeated the San Francisco 49ers with a final score of 25-22.
